# INFOMDWR – Assignment 2: Data Integration & Preparation

## Introduction
In this assignment, you will work on different tasks that are related to data preparation including data profiling, finding records that refer to same entity, and computing the correlation between different attributes. The datasets that you need to work on are available online (links are provided).

In [ ]:
# Import the necessary libraries

from typing import Any
import pandas as pd
import numpy as np
from py_stringmatching import similarity_measure as sm

In [143]:
# Load the csv into a dataframe and print a part of the dataframe

df = pd.read_csv("dft-road-casualty-statistics-collision-2024.csv", low_memory = False)
df

,collision_index,collision_year,collision_ref_no,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,collision_severity,number_of_vehicles,...,carriageway_hazards_historic,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,enhanced_severity_collision,collision_injury_based,collision_adjusted_severity_serious,collision_adjusted_severity_slight
0,202417H103224,2024,17H103224,448894,532505,-1.24312,54.68523,17,3,2,...,-1,0,1,2,2,E01011983,3,1,0.000000,1.000000
1,202417M217924,2024,17M217924,452135,519436,-1.19517,54.56747,17,2,2,...,-1,0,1,3,2,E01012061,7,1,1.000000,0.000000
2,202417S204524,2024,17S204524,445427,522924,-1.29837,54.59946,17,3,2,...,0,0,2,1,2,E01012280,-1,0,0.111621,0.888379
3,2024481510889,2024,481510889,533587,181174,-0.07626,51.51371,48,2,1,...,-1,0,1,1,2,E01000005,7,1,1.000000,0.000000
4,2024481563500,2024,481563500,532676,180902,-0.08948,51.51148,48,2,1,...,-1,0,1,1,2,E01032739,5,1,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100922,2024991432330,2024,991432330,255558,666585,-4.31001,55.87072,99,2,1,...,-1,0,1,1,-1,-1,6,1,1.000000,0.000000
100923,2024991466055,2024,991466055,258265,666632,-4.26681,55.87194,99,3,1,...,-1,0,1,1,-1,-1,3,1,0.000000,1.000000
100924,2024991485880,2024,991485880,264189,664898,-4.17134,55.85808,99,2,2,...,-1,0,1,1,-1,-1,7,1,1.000000,0.000000
100925,2024991436758,2024,991436758,291004,658589,-3.74065,55.80823,99,3,2,...,-1,0,2,1,-1,-1,3,1,0.000000,1.000000


## Task 1: Profiling relational data

For this task, download and read the paper about profiling relational data, select a set of summary statistics about the data (minimum of 10 different values) and write Python code to compute these quantities for a dataset of your choice. Preferably, you can use one of the csv files from the road safety dataset. Explain the importance of each summary statistic that you selected in understanding the characteristics of the dataset.

*Note: Computing the same statistical quantity on multiple columns of the dataset will be counted only once.*

In [144]:
# Set up a dictionary to hold the summary statistics

summary_statistics: dict[str, Any] = {
    "mean": None,
    "median": None,
    "mode": None,
    "minimum": None,
    "maximum": None,
    "range": None,
    "variance": None,
    "standard_deviation": None,
    "records": None,
    "number_of_missing_records": None,
}

In [145]:
# Fill the dictionary for summary statistics

summary_statistics["mean"] = df["speed_limit"].mean()                                           # Mean of the speed limit
summary_statistics["median"] = df["speed_limit"].median()                                       # Median of the speed limit 
summary_statistics["mode"] = df["speed_limit"].mode()                                           # Mode of the speed limit
summary_statistics["minimum"] = df["speed_limit"].min()                                         # Min of the speed limit
summary_statistics["maximum"] = df["speed_limit"].max()                                         # Max of the speed limit
summary_statistics["range"] = abs(df["speed_limit"].max()) + abs(df["speed_limit"].min())       # Range of the speed limit, thus abs(max)+ and(min))
summary_statistics["variance"] = df["speed_limit"].var()                                        # Variance of the speed limit
summary_statistics["standard_deviation"] = df["speed_limit"].std()                              # Standard deviation of the speed limit
summary_statistics["records"] = len(df["speed_limit"])                                          # Number of rows in the dataframe
summary_statistics["number_of_missing_records"] = df.isnull().sum().sum()                       # Number of missing values in the entire dataframe

summary_statistics

{'mean': np.float64(35.87966550080751),
 'median': np.float64(30.0),
 'mode': 0    30
 Name: speed_limit, dtype: int64,
 'minimum': np.int64(-1),
 'maximum': np.int64(70),
 'range': np.int64(71),
 'variance': np.float64(211.30421831349014),
 'standard_deviation': np.float64(14.536306900774012),
 'records': 100927,
 'number_of_missing_records': np.int64(3)}

## Task 2: Entity resolution

### Part 1:

Write a Python code to compare every single record in the dataset (ACM.csv) with all the records in (DBLP2.csv) and find the similar records (records that represent the same publication). To compare two records, follow the steps:

In [146]:
# Import the datasets

df_acm = pd.read_csv("ACM.csv")
df_dblp2 = pd.read_csv("DBLP2.csv", encoding="latin1")


In [147]:
# Show the dataframe of ACM

df_acm

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
2291,672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994
2292,945741,Review of The data warehouse toolkit: the comp...,Alexander A. Anisimov,ACM SIGMOD Record,2003


In [148]:
# Show the dataframe of dblp2

df_dblp2

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,A simple algorithm for finding frequent elemen...,"Scott Shenker, Christos H. Papadimitriou, Rich...",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynami...,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for ...,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael ...",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,Chair's Message,Richard T. Snodgrass,SIGMOD Record,2001


## Part 1

### A. 

Ignore the pub_id

In [149]:
df_acm.drop("id", axis=1, inplace = True)

In [150]:
df_dblp2.drop("id", axis=1, inplace = True)

### B. 

Change all alphabetical characters into lowercase.

In [151]:
for col in df_acm.columns:
    df_acm[col] = df_acm[col].astype(str).str.lower()
for col in df_dblp2.columns:
    df_dblp2[col] = df_dblp2[col].astype(str).str.lower()


In [152]:
df_acm

,title,authors,venue,year
0,the wasa2 object-oriented workflow management ...,"gottfried vossen, mathias weske",international conference on management of data,1999
1,a user-centered interface for querying distrib...,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,"world wide database-integrating the web, corba...","athman bouguettaya, boualem benatallah, lily h...",international conference on management of data,1999
3,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228...",international conference on management of data,1999
4,the ccube constraint object-oriented database ...,"alexander brodsky, victor e. segal, jia chen, ...",international conference on management of data,1999
...,...,...,...,...
2289,dual-buffering strategies in object bases,"alfons kemper, donald kossmann",very large data bases,1994
2290,guest editorial,"philip a. bernstein, yannis ioannidis, raghu r...",the vldb journal &mdash; the international jou...,2003
2291,graphdb: modeling and querying graphs in datab...,ralf hartmut g&#252;ting,very large data bases,1994
2292,review of the data warehouse toolkit: the comp...,alexander a. anisimov,acm sigmod record,2003


In [153]:
df_dblp2

,title,authors,venue,year
0,semantic integration of environmental models f...,d. scott mackay,sigmod record,1999
1,estimation of query-result distribution and it...,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,incremental maintenance for non-distributive a...,"themistoklis palpanas, richard sidle, hamid pi...",vldb,2002
3,cost-based selection of path expression proces...,"zhao-hui tang, georges gardarin, jean-robert g...",vldb,1996
4,benchmarking spatial join operations with spat...,"erik g. hoel, hanan samet",vldb,1995
...,...,...,...,...
2611,a simple algorithm for finding frequent elemen...,"scott shenker, christos h. papadimitriou, rich...",acm trans. database syst.,2003
2612,sash: a self-adaptive histogram set for dynami...,"lipyeow lim, min wang, jeffrey scott vitter",vldb,2003
2613,locally adaptive dimensionality reduction for ...,"kaushik chakrabarti, eamonn j. keogh, michael ...",acm trans. database syst.,2002
2614,chair's message,richard t. snodgrass,sigmod record,2001


### C. 

Convert multiple spaces to one

In [1]:
for col in df_acm.columns:
    df_acm[col].replace(r'\s {2,}', ' ', regex=True)

df_acm

NameError: name 'df_acm' is not defined

In [170]:
for col in df_dblp2.columns:
    df_dblp2[col].replace(r'\s {2,}', ' ', regex=True)

df_dblp2

,title,authors,venue,year
0,semantic integration of environmental models f...,d. scott mackay,sigmod record,1999
1,estimation of query-result distribution and it...,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,incremental maintenance for non-distributive a...,"themistoklis palpanas, richard sidle, hamid pi...",vldb,2002
3,cost-based selection of path expression proces...,"zhao-hui tang, georges gardarin, jean-robert g...",vldb,1996
4,benchmarking spatial join operations with spat...,"erik g. hoel, hanan samet",vldb,1995
...,...,...,...,...
2611,a simple algorithm for finding frequent elemen...,"scott shenker, christos h. papadimitriou, rich...",acm trans. database syst.,2003
2612,sash: a self-adaptive histogram set for dynami...,"lipyeow lim, min wang, jeffrey scott vitter",vldb,2003
2613,locally adaptive dimensionality reduction for ...,"kaushik chakrabarti, eamonn j. keogh, michael ...",acm trans. database syst.,2002
2614,chair's message,richard t. snodgrass,sigmod record,2001


### D.

Use Levenshtein similarity $(L_{sim}(S_1, S_2) = 1 - \frac{MED(S_1, S_2}{MAX(|S_1|, |S_2|})$
for comparing the values in the **title** attribute and compute the score $(s_t)$. (MED refers to the minimum edit distance and $|S_i|$ is the number of characters in string $S_i$).


### E. 

Use Jaro similarity to compare the values in the **authors** field and compute $(S_a)$.

### F.

Use a modified version of the affine similarity that is scaled to the interval [0, 1] for the venue attribute $(S_c)$.

### G.

Use Match (1) / Mismatch (0) for the year $(S_y)$.

### H.

Use the formula $\text{rec\_sim} = w_1 * s_1 + w_2 * s_2 + w_3 * s_3 + w_4 * s_4$ to combine the scores and compute the final score, where $\sum^4_{i=1}w_i=1$.

### I.

Report the records with rec_sim > 0.7 as duplicate records by storing the ids of both records in a list

### J.

In the table `DBLP-ACM_perfectMapping.csv`, you can find the actual mappings (the ids of the correct duplicate records). Compute the precision of this method by counting the number of duplicate records that you discovered correctly. That is, among all the reported similar records by your method, how many pairs exist in the file `DBLP-ACM_perfectMapping.csv`

### K.

Record the running time of the method. You can observe that the program takes a long time to get the results. What can you do to reduce the running time? (Just provide clear discussion – no need for implementing the ideas.)

## Part 2

### 1.

Concatenate the values in each record into one single string.

### 2.

Change all alphabetical characters into lowercase.

### 3.

Convert multiple spaces to one.

### 4.

Combine the records from both tables into one big list as we did during the lab.

### 5.

Use the functions in the tutorials from lab 5 to compute the shingles, the minhash signature and the similarity.

### 6.

Extract the top 2224 candidates from the LSH algorithm, compare them to the actual mappings in the file `DBLP-ACM_perfectMapping.csv` and compute the precision of the method.

### 7.

Record the running time of the method.

### 8. 

Compare the precision and the running time in Parts 1 and 2.

## Task 3. Data preperation

For this task, use the Pima Indians Diabetes Database.

### 1. 

Compute the correlation between the different columns after removing the `outcome` column.

### 2.

Remove the disguised values from the table. We need to remove the values that equal to `0` from columns `BloodPressure`, `SkinThickness` and `BMI` as these are missing values but they have been replaced by the value `0`. Remove the value but keep the record (i.e.) change the value to `null`.

### 3.

Fill the cells with `null` using the mean values of the records that have the same class label.

### 4.

Compute the correlation between the different columns.

### 5.

Compare the values from this step with the values in the first step (just mention the most important changes (if any)) and comment on your findings.